Ce notebook vise à traiter les données issues de ERA5 (au dessus du BF ou des mers). Toutes les variables seront mises en commun et sauvegardées dans deux fichiers (decadaires et mensuelles)

In [1]:
import os, sys
import pyproj
from pyproj import Proj 
from netCDF4 import Dataset
import netCDF4
import shutil
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import datetime
import cftime
import subprocess
import shutil
from functools import reduce
from config import Config, Constant
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Lecture du fichier des données des coordonnées des stations
#Ce fichier permet d'extraire les valeurs au dessus des stations à partir des fichiers NetCDF
stations = pd.read_excel(str(Config.STATION_PATH) + '/' + str(Config.STAT_COORD_FILE_NAME))

In [3]:
# Liste contenant les noms des dataframes decades et mensuels
df_terre_dek=[]
df_terre_mon=[]
df_mer_dek=[]
df_mer_mon=[]

### I- VITESSE VENT V À LA 975 hPa

In [4]:
# Lecture de fichier de vent v à la 975hPa
v_wind_975=Dataset(str(Config.ERA5_PATH) + '/' + str(Config.WIND_V_975))
#Lecture des variables du fichier au format netCDF
v_wind=v_wind_975.variables['v'][:]
unitd=v_wind_975.variables['v'].units
unit=v_wind_975.variables['time'].units
times=v_wind_975.variables['time'][:]
lat=v_wind_975.variables['latitude'][:]
lon=v_wind_975.variables['longitude'][:]

In [5]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_out = Config.ERA5_DATA_PATH
os.makedirs(dir_out, exist_ok=True)

# Créer une liste pour stocker les informations des stations
station_data_list = []

for index, value in stations.iterrows():
    station = value['station']  
    
    lon_point = value['longitude']  # Assurez-vous que la colonne s'appelle 'Longitude'
    lat_point = value['latitude']   # Assurez-vous que la colonne s'appelle 'Latitude'
    
    # Calcul des carrés des différences entre les latitudes et longitudes
    sq_diff_lat = (lat - lat_point) ** 2
    sq_diff_lon = (lon - lon_point) ** 2
    
    print(f"Generating time series for {station}")
    ref_date = datetime.datetime(int(unit[12:16]), int(unit[17:19]), int(unit[20:22]))
    date_range = []
    v_wind_data = []
    
    for index, time in enumerate(times):
        date_time = ref_date + datetime.timedelta(hours=int(time))
        date_range.append(date_time)  # Ajoutez l'objet datetime lui-même
        v_wind_data.append(v_wind[index, sq_diff_lat.argmin(), sq_diff_lon.argmin()])  # Utilisez les indices corrects
    station_data_list.append({'Station': station, 'Date-Time': date_range, 'v_wind_975': v_wind_data})

# Création du DataFrame global
all_data = pd.DataFrame()

for station_data in station_data_list:
    df = pd.DataFrame(station_data)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Enregistrement du DataFrame global dans un fichier CSV
output_file_path = os.path.join(dir_out, str(Config.WIND_V_975_CSV))
all_data.to_csv(output_file_path, index=False)
print(f"All station data is saved in the CSV file: {output_file_path}")


Generating time series for Bobo_Dioulasso
Generating time series for Bogande
Generating time series for Boromo
Generating time series for Dedougou
Generating time series for Dori
Generating time series for Fada_Ngourma
Generating time series for Gaoua
Generating time series for Ouagadougou_aero
Generating time series for Ouahigouya
Generating time series for Po
All station data is saved in the CSV file: ../ASSETS/DATA/ERA5/v_wind_975.csv


In [6]:
all_data.head()

,Station,Date-Time,v_wind_975
0,Bobo_Dioulasso,1961-01-01 00:00:00,-2.880353
1,Bobo_Dioulasso,1961-01-01 06:00:00,-2.466383
2,Bobo_Dioulasso,1961-01-01 12:00:00,-1.585962
3,Bobo_Dioulasso,1961-01-01 18:00:00,-2.316918
4,Bobo_Dioulasso,1961-01-02 00:00:00,-2.920239


In [7]:
# Supposons que vous ayez déjà chargé vos données dans un DataFrame appelé 'df'
# Assurez-vous que la colonne 'Date-Time' est en format de datetime
df=all_data.copy()
df['Date-Time'] = pd.to_datetime(df['Date-Time'])
# Créez une colonne 'Decade' en utilisant le numéro de décade dans le mois
decade_ranges = [0, 10, 20, 32]
dek_num = [1, 2, 3]
df['Day']=df['Date-Time'].dt.day
df['Decade'] = pd.cut(df['Day'], bins=decade_ranges, labels=dek_num)
# Créez les colonnes 'Year' et 'Month' à partir de la colonne 'Date-Time'
df['Year'] = df['Date-Time'].dt.year
df['Month'] = df['Date-Time'].dt.month


##### I-1 CALCUL DES MOYENNES DECADAIRES 

In [8]:
# Regroupez par année, mois et décade, puis calculez la moyenne décadaires pour 'd2m (m s**-1)'
decade_avg_u_w_975 = df.groupby(['Station','Year', 'Month', 'Decade'])['v_wind_975'].mean()
# Réinitialisez l'index pour obtenir une DataFrame propre avec les valeurs calculées
decade_avg_u_w_975 = decade_avg_u_w_975.reset_index()
#output_file_path = os.path.join(dir_out, str(Config.WIND_V_975_CSV))
#output_dek_path = os.path.join(dir_out, str(Config.WIND_V_975_DEK))
#decade_avg_u_w_975.to_csv(output_dek_path, index=False)
#Ajout du nom de la dataframe pour la concatenation
df_terre_dek.append(decade_avg_u_w_975)
# Affichez les moyennes décadaires
print("Moyenne décadaire :\n", decade_avg_u_w_975)

Moyenne décadaire :
               Station  Year  Month Decade  v_wind_975
0      Bobo_Dioulasso  1961      1      1   -2.477929
1      Bobo_Dioulasso  1961      1      2   -2.545220
2      Bobo_Dioulasso  1961      1      3   -2.898044
3      Bobo_Dioulasso  1961      2      1   -2.424871
4      Bobo_Dioulasso  1961      2      2   -2.312341
...               ...   ...    ...    ...         ...
21955              Po  2021     11      2   -0.261836
21956              Po  2021     11      3   -0.620669
21957              Po  2021     12      1   -1.527897
21958              Po  2021     12      2   -2.103897
21959              Po  2021     12      3   -3.658856

[21960 rows x 5 columns]


##### I-2 CALCUL DES MOYENNES MENSUELLES

In [9]:
# Maintenant, calculez la moyenne mensuelle en utilisant le même processus
month_avg_u_w_975 = df.groupby(['Station','Year', 'Month'])['v_wind_975'].mean()
month_avg_u_w_975 = month_avg_u_w_975.reset_index()
# Affichez les moyennes mensuelles
print("\nMoyenne mensuelle :\n", month_avg_u_w_975)
#output_mon_path = os.path.join(dir_out, str(Config.WIND_V_975_MON))
#month_avg_u_w_975.to_csv(output_mon_path, index=False)
df_terre_mon.append(month_avg_u_w_975)


Moyenne mensuelle :
              Station  Year  Month  v_wind_975
0     Bobo_Dioulasso  1961      1   -2.648709
1     Bobo_Dioulasso  1961      2   -2.103444
2     Bobo_Dioulasso  1961      3   -0.886015
3     Bobo_Dioulasso  1961      4    1.652940
4     Bobo_Dioulasso  1961      5    1.654241
...              ...   ...    ...         ...
7315              Po  2021      8    1.879763
7316              Po  2021      9    2.036198
7317              Po  2021     10    1.988795
7318              Po  2021     11    0.455663
7319              Po  2021     12   -2.469851

[7320 rows x 4 columns]


### II- COMPOSANTE U DU VENT À LA 700 hPa

In [10]:
# Lecture de fichier de vent u à la 700 hPa
u_wind_700=Dataset(str(Config.ERA5_PATH) + '/' + str(Config.WIND_U_700))
#Lecture des variables du fichier au format netCDF
u700_wind=u_wind_700.variables['u'][:]
unitd=u_wind_700.variables['u'].units
unit=u_wind_700.variables['time'].units
times=u_wind_700.variables['time'][:]
lat=u_wind_700.variables['latitude'][:]
lon=u_wind_700.variables['longitude'][:]

In [11]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_out = Config.ERA5_DATA_PATH
os.makedirs(dir_out, exist_ok=True)

# Créer une liste pour stocker les informations des stations
station_data_list = []

for index, value in stations.iterrows():
    station = value['station']  
    
    lon_point = value['longitude']  # Assurez-vous que la colonne s'appelle 'Longitude'
    lat_point = value['latitude']   # Assurez-vous que la colonne s'appelle 'Latitude'
    
    # Calcul des carrés des différences entre les latitudes et longitudes
    sq_diff_lat = (lat - lat_point) ** 2
    sq_diff_lon = (lon - lon_point) ** 2
    
    print(f"Generating time series for {station}")
    ref_date = datetime.datetime(int(unit[12:16]), int(unit[17:19]), int(unit[20:22]))
    date_range = []
    u700_wind_data = []
    
    for index, time in enumerate(times):
        date_time = ref_date + datetime.timedelta(hours=int(time))
        date_range.append(date_time)  # Ajoutez l'objet datetime lui-même
        u700_wind_data.append(u700_wind[index, sq_diff_lat.argmin(), sq_diff_lon.argmin()])  # Utilisez les indices corrects
    station_data_list.append({'Station': station, 'Date-Time': date_range, 'u_wind_700': u700_wind_data})

# Création du DataFrame global
all_data = pd.DataFrame()

for station_data in station_data_list:
    df = pd.DataFrame(station_data)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Enregistrement du DataFrame global dans un fichier CSV
output_file_path = os.path.join(dir_out, str(Config.WIND_U_700_CSV))
all_data.to_csv(output_file_path, index=False)
print(f"All station data is saved in the CSV file: {output_file_path}")


Generating time series for Bobo_Dioulasso
Generating time series for Bogande
Generating time series for Boromo
Generating time series for Dedougou
Generating time series for Dori
Generating time series for Fada_Ngourma
Generating time series for Gaoua
Generating time series for Ouagadougou_aero
Generating time series for Ouahigouya
Generating time series for Po
All station data is saved in the CSV file: ../ASSETS/DATA/ERA5/u_wind_700.csv


In [12]:
all_data.head()

,Station,Date-Time,u_wind_700
0,Bobo_Dioulasso,1961-01-01 00:00:00,-3.787889
1,Bobo_Dioulasso,1961-01-01 06:00:00,-4.108588
2,Bobo_Dioulasso,1961-01-01 12:00:00,-3.107422
3,Bobo_Dioulasso,1961-01-01 18:00:00,-2.434280
4,Bobo_Dioulasso,1961-01-02 00:00:00,-5.467894


In [13]:
# Supposons que vous ayez déjà chargé vos données dans un DataFrame appelé 'df'
# Assurez-vous que la colonne 'Date-Time' est en format de datetime
df=all_data.copy()
df['Date-Time'] = pd.to_datetime(df['Date-Time'])
# Créez une colonne 'Decade' en utilisant le numéro de décade dans le mois
decade_ranges = [0, 10, 20, 32]
dek_num = [1, 2, 3]
df['Day']=df['Date-Time'].dt.day
df['Decade'] = pd.cut(df['Day'], bins=decade_ranges, labels=dek_num)
# Créez les colonnes 'Year' et 'Month' à partir de la colonne 'Date-Time'
df['Year'] = df['Date-Time'].dt.year
df['Month'] = df['Date-Time'].dt.month

##### II-1 CALCUL DES MOYENNES DECADAIRES 

In [14]:
# Regroupez par année, mois et décade, puis calculez la moyenne décadaires pour 'd2m (m s**-1)'
decade_avg_u_w_700 = df.groupby(['Station','Year', 'Month', 'Decade'])['u_wind_700'].mean()
# Réinitialisez l'index pour obtenir une DataFrame propre avec les valeurs calculées
decade_avg_u_w_700 = decade_avg_u_w_700.reset_index()
#output_file_path = os.path.join(dir_out, str(Config.WIND_V_975_CSV))
#output_dek_path = os.path.join(dir_out, str(Config.WIND_U_700_DEK))
#decade_avg_u_w_700.to_csv(output_dek_path, index=False)

# Affichez les moyennes décadaires
print("Moyenne décadaire :\n", decade_avg_u_w_700)
df_terre_dek.append(decade_avg_u_w_700)

Moyenne décadaire :
               Station  Year  Month Decade  u_wind_700
0      Bobo_Dioulasso  1961      1      1   -6.607575
1      Bobo_Dioulasso  1961      1      2   -1.387879
2      Bobo_Dioulasso  1961      1      3    2.169399
3      Bobo_Dioulasso  1961      2      1    3.171007
4      Bobo_Dioulasso  1961      2      2    3.065763
...               ...   ...    ...    ...         ...
21955              Po  2021     11      2   -5.803122
21956              Po  2021     11      3   -8.759756
21957              Po  2021     12      1   -4.211920
21958              Po  2021     12      2   -5.516121
21959              Po  2021     12      3   -1.577647

[21960 rows x 5 columns]


##### II-2 CALCUL DES MOYENNES MENSUELLES

In [15]:
# Maintenant, calculez la moyenne mensuelle en utilisant le même processus
month_avg_u_w_700 = df.groupby(['Station','Year', 'Month'])['u_wind_700'].mean()
month_avg_u_w_700 = month_avg_u_w_700.reset_index()
# Affichez les moyennes mensuelles
print("\nMoyenne mensuelle :\n", month_avg_u_w_700)
#output_mon_path = os.path.join(dir_out, str(Config.WIND_U_700_MON))
#month_avg_u_w_700.to_csv(output_mon_path, index=False)
df_terre_mon.append(month_avg_u_w_700)


Moyenne mensuelle :
              Station  Year  Month  u_wind_700
0     Bobo_Dioulasso  1961      1   -1.809392
1     Bobo_Dioulasso  1961      2    0.458890
2     Bobo_Dioulasso  1961      3   -1.004652
3     Bobo_Dioulasso  1961      4   -4.718261
4     Bobo_Dioulasso  1961      5   -8.676686
...              ...   ...    ...         ...
7315              Po  2021      8   -6.685376
7316              Po  2021      9  -11.709194
7317              Po  2021     10  -10.225080
7318              Po  2021     11   -7.267599
7319              Po  2021     12   -3.697888

[7320 rows x 4 columns]


### III- COMPOSANTE U DU VENT À LA 100 hPa

In [16]:
# Lecture de fichier de vent u à la 700 hPa
u_wind_100=Dataset(str(Config.ERA5_PATH) + '/' + str(Config.WIND_U_100))
#Lecture des variables du fichier au format netCDF
u100_wind=u_wind_100.variables['u'][:]
unitd=u_wind_100.variables['u'].units
unit=u_wind_100.variables['time'].units
times=u_wind_100.variables['time'][:]
lat=u_wind_100.variables['latitude'][:]
lon=u_wind_100.variables['longitude'][:]

In [17]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_out = Config.ERA5_DATA_PATH
os.makedirs(dir_out, exist_ok=True)

# Créer une liste pour stocker les informations des stations
station_data_list = []

for index, value in stations.iterrows():
    station = value['station']  
    
    lon_point = value['longitude']  # Assurez-vous que la colonne s'appelle 'Longitude'
    lat_point = value['latitude']   # Assurez-vous que la colonne s'appelle 'Latitude'
    
    # Calcul des carrés des différences entre les latitudes et longitudes
    sq_diff_lat = (lat - lat_point) ** 2
    sq_diff_lon = (lon - lon_point) ** 2
    
    print(f"Generating time series for {station}")
    ref_date = datetime.datetime(int(unit[12:16]), int(unit[17:19]), int(unit[20:22]))
    date_range = []
    u100_wind_data = []
    
    for index, time in enumerate(times):
        date_time = ref_date + datetime.timedelta(hours=int(time))
        date_range.append(date_time)  # Ajoutez l'objet datetime lui-même
        u100_wind_data.append(u100_wind[index, sq_diff_lat.argmin(), sq_diff_lon.argmin()])  # Utilisez les indices corrects
    station_data_list.append({'Station': station, 'Date-Time': date_range, 'u_wind_100': u100_wind_data})

# Création du DataFrame global
all_data = pd.DataFrame()

for station_data in station_data_list:
    df = pd.DataFrame(station_data)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Enregistrement du DataFrame global dans un fichier CSV
output_file_path = os.path.join(dir_out, str(Config.WIND_U_100_CSV))
all_data.to_csv(output_file_path, index=False)
print(f"All station data is saved in the CSV file: {output_file_path}")


Generating time series for Bobo_Dioulasso
Generating time series for Bogande
Generating time series for Boromo
Generating time series for Dedougou
Generating time series for Dori
Generating time series for Fada_Ngourma
Generating time series for Gaoua
Generating time series for Ouagadougou_aero
Generating time series for Ouahigouya
Generating time series for Po
All station data is saved in the CSV file: ../ASSETS/DATA/ERA5/u_wind_100.csv


In [18]:
all_data.head()

,Station,Date-Time,u_wind_100
0,Bobo_Dioulasso,1961-01-01 00:00:00,13.119489
1,Bobo_Dioulasso,1961-01-01 06:00:00,12.816139
2,Bobo_Dioulasso,1961-01-01 12:00:00,13.254617
3,Bobo_Dioulasso,1961-01-01 18:00:00,17.120947
4,Bobo_Dioulasso,1961-01-02 00:00:00,19.713208


In [19]:
# Supposons que vous ayez déjà chargé vos données dans un DataFrame appelé 'df'
# Assurez-vous que la colonne 'Date-Time' est en format de datetime
df=all_data.copy()
df['Date-Time'] = pd.to_datetime(df['Date-Time'])
# Créez une colonne 'Decade' en utilisant le numéro de décade dans le mois
decade_ranges = [0, 10, 20, 32]
dek_num = [1, 2, 3]
df['Day']=df['Date-Time'].dt.day
df['Decade'] = pd.cut(df['Day'], bins=decade_ranges, labels=dek_num)
# Créez les colonnes 'Year' et 'Month' à partir de la colonne 'Date-Time'
df['Year'] = df['Date-Time'].dt.year
df['Month'] = df['Date-Time'].dt.month

##### III-1 CALCUL DES MOYENNES DECADAIRES 

In [20]:
# Regroupez par année, mois et décade, puis calculez la moyenne décadaires pour 'd2m (m s**-1)'
decade_avg_u_w_100 = df.groupby(['Station','Year', 'Month', 'Decade'])['u_wind_100'].mean()
# Réinitialisez l'index pour obtenir une DataFrame propre avec les valeurs calculées
decade_avg_u_w_100 = decade_avg_u_w_100.reset_index()
#output_file_path = os.path.join(dir_out, str(Config.WIND_V_975_CSV))
#output_dek_path = os.path.join(dir_out, str(Config.WIND_U_100_DEK))
#decade_avg_u_w_100.to_csv(output_dek_path, index=False)

# Affichez les moyennes décadaires
print("Moyenne décadaire :\n", decade_avg_u_w_100)
df_terre_dek.append(decade_avg_u_w_100)

Moyenne décadaire :
               Station  Year  Month Decade  u_wind_100
0      Bobo_Dioulasso  1961      1      1   16.585018
1      Bobo_Dioulasso  1961      1      2   18.501326
2      Bobo_Dioulasso  1961      1      3   17.749175
3      Bobo_Dioulasso  1961      2      1    9.010893
4      Bobo_Dioulasso  1961      2      2   11.091734
...               ...   ...    ...    ...         ...
21955              Po  2021     11      2    7.834724
21956              Po  2021     11      3    5.852299
21957              Po  2021     12      1   11.448239
21958              Po  2021     12      2   18.516873
21959              Po  2021     12      3   17.013145

[21960 rows x 5 columns]


##### III-2 CALCUL DES MOYENNES MENSUELLES

In [21]:
# Maintenant, calculez la moyenne mensuelle en utilisant le même processus
month_avg_u_w_100 = df.groupby(['Station','Year', 'Month'])['u_wind_100'].mean()
month_avg_u_w_100 = month_avg_u_w_100.reset_index()
# Affichez les moyennes mensuelles
print("\nMoyenne mensuelle :\n", month_avg_u_w_100)
output_mon_path = os.path.join(dir_out, str(Config.WIND_U_100_MON))
month_avg_u_w_100.to_csv(output_mon_path, index=False)
df_terre_mon.append(month_avg_u_w_100)


Moyenne mensuelle :
              Station  Year  Month  u_wind_100
0     Bobo_Dioulasso  1961      1   17.616270
1     Bobo_Dioulasso  1961      2   12.566036
2     Bobo_Dioulasso  1961      3   16.747609
3     Bobo_Dioulasso  1961      4    5.372466
4     Bobo_Dioulasso  1961      5   -4.629670
...              ...   ...    ...         ...
7315              Po  2021      8  -15.133298
7316              Po  2021      9   -8.326725
7317              Po  2021     10   -2.909251
7318              Po  2021     11    7.878928
7319              Po  2021     12   15.703087

[7320 rows x 4 columns]


### IV- EAU PRECIPITABLE

In [22]:
# Lecture de fichier de vent u à la 700 hPa
eau_precipitable=Dataset(str(Config.ERA5_PATH) + '/' + str(Config.EAU_PREC))
#Lecture des variables du fichier au format netCDF
eau_prec=eau_precipitable.variables['p79.162'][:]
unitd=eau_precipitable.variables['p79.162'].units
unit=eau_precipitable.variables['time'].units
times=eau_precipitable.variables['time'][:]
lat=eau_precipitable.variables['latitude'][:]
lon=eau_precipitable.variables['longitude'][:]
#expv=eau_precipitable.variables['expver'][:]

In [23]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_out = Config.ERA5_DATA_PATH
os.makedirs(dir_out, exist_ok=True)

# Créer une liste pour stocker les informations des stations
station_data_list = []

for index, value in stations.iterrows():
    station = value['station']  
    
    lon_point = value['longitude']  # Assurez-vous que la colonne s'appelle 'Longitude'
    lat_point = value['latitude']   # Assurez-vous que la colonne s'appelle 'Latitude'
    
    # Calcul des carrés des différences entre les latitudes et longitudes
    sq_diff_lat = (lat - lat_point) ** 2
    sq_diff_lon = (lon - lon_point) ** 2
    
    print(f"Generating time series for {station}")
    ref_date = datetime.datetime(int(unit[12:16]), int(unit[17:19]), int(unit[20:22]))
    date_range = []
    eau_prec_data = []
    
    for index, time in enumerate(times):
        date_time = ref_date + datetime.timedelta(hours=int(time))
        date_range.append(date_time)  # Ajoutez l'objet datetime lui-même
        eau_prec_data.append(eau_prec[index, sq_diff_lat.argmin(), sq_diff_lon.argmin()])  #expv[1] Utilisez les indices corrects
    station_data_list.append({'Station': station, 'Date-Time': date_range, 'eau_precipitable': eau_prec_data})

# Création du DataFrame global
all_data = pd.DataFrame()

for station_data in station_data_list:
    df = pd.DataFrame(station_data)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Enregistrement du DataFrame global dans un fichier CSV
output_file_path = os.path.join(dir_out, str(Config.EAU_PREC_CSV))
all_data.to_csv(output_file_path, index=False)
print(f"All station data is saved in the CSV file: {output_file_path}")

Generating time series for Bobo_Dioulasso
Generating time series for Bogande
Generating time series for Boromo
Generating time series for Dedougou
Generating time series for Dori
Generating time series for Fada_Ngourma
Generating time series for Gaoua
Generating time series for Ouagadougou_aero
Generating time series for Ouahigouya
Generating time series for Po
All station data is saved in the CSV file: ../ASSETS/DATA/ERA5/eau_prec.csv


In [24]:
all_data.head()

,Station,Date-Time,eau_precipitable
0,Bobo_Dioulasso,1961-01-01 00:00:00,-3.419611e-07
1,Bobo_Dioulasso,1961-01-01 06:00:00,-1.778531e-06
2,Bobo_Dioulasso,1961-01-01 12:00:00,-2.240338e-07
3,Bobo_Dioulasso,1961-01-01 18:00:00,1.511897e-07
4,Bobo_Dioulasso,1961-01-02 00:00:00,-6.957432e-07


In [25]:
# Supposons que vous ayez déjà chargé vos données dans un DataFrame appelé 'df'
# Assurez-vous que la colonne 'Date-Time' est en format de datetime
df=all_data.copy()
df['Date-Time'] = pd.to_datetime(df['Date-Time'])
# Créez une colonne 'Decade' en utilisant le numéro de décade dans le mois
decade_ranges = [0, 10, 20, 32]
dek_num = [1, 2, 3]
df['Day']=df['Date-Time'].dt.day
df['Decade'] = pd.cut(df['Day'], bins=decade_ranges, labels=dek_num)
# Créez les colonnes 'Year' et 'Month' à partir de la colonne 'Date-Time'
df['Year'] = df['Date-Time'].dt.year
df['Month'] = df['Date-Time'].dt.month

##### IV-1 CALCUL DES MOYENNES DECADAIRES 

In [26]:
# Regroupez par année, mois et décade, puis calculez la moyenne décadaires pour 'd2m (m s**-1)'
decade_avg_eau_prec = df.groupby(['Station','Year', 'Month', 'Decade'])['eau_precipitable'].mean()
# Réinitialisez l'index pour obtenir une DataFrame propre avec les valeurs calculées
decade_avg_eau_prec = decade_avg_eau_prec.reset_index()
#output_file_path = os.path.join(dir_out, str(Config.WIND_V_975_CSV))
output_dek_path = os.path.join(dir_out, str(Config.EAU_PREC_CSV_DEK))
decade_avg_eau_prec.to_csv(output_dek_path, index=False)

# Affichez les moyennes décadaires
print("Moyenne décadaire :\n", decade_avg_eau_prec)
df_terre_dek.append(decade_avg_eau_prec)

Moyenne décadaire :
               Station  Year  Month Decade  eau_precipitable
0      Bobo_Dioulasso  1961      1      1      7.668101e-08
1      Bobo_Dioulasso  1961      1      2     -2.891759e-08
2      Bobo_Dioulasso  1961      1      3     -8.290206e-07
3      Bobo_Dioulasso  1961      2      1     -4.955488e-08
4      Bobo_Dioulasso  1961      2      2     -1.390865e-08
...               ...   ...    ...    ...               ...
21955              Po  2021     11      2     -2.084888e-07
21956              Po  2021     11      3     -7.689258e-08
21957              Po  2021     12      1      4.907528e-08
21958              Po  2021     12      2     -1.235275e-07
21959              Po  2021     12      3      4.267757e-09

[21960 rows x 5 columns]


##### IV-2 CALCUL DES MOYENNES MENSUELLES

In [27]:
# Maintenant, calculez la moyenne mensuelle en utilisant le même processus
month_avg_eau_prec = df.groupby(['Station','Year', 'Month'])['eau_precipitable'].mean()
month_avg_eau_prec = month_avg_eau_prec.reset_index()
# Affichez les moyennes mensuelles
print("\nMoyenne mensuelle :\n", month_avg_eau_prec)
output_mon_path = os.path.join(dir_out, str(Config.EAU_PREC_CSV_MON))
month_avg_eau_prec.to_csv(output_mon_path, index=False)
df_terre_mon.append(month_avg_eau_prec)


Moyenne mensuelle :
              Station  Year  Month  eau_precipitable
0     Bobo_Dioulasso  1961      1     -2.787611e-07
1     Bobo_Dioulasso  1961      2     -4.130379e-08
2     Bobo_Dioulasso  1961      3      2.225167e-07
3     Bobo_Dioulasso  1961      4      8.248803e-08
4     Bobo_Dioulasso  1961      5     -1.169135e-07
...              ...   ...    ...               ...
7315              Po  2021      8     -1.110345e-07
7316              Po  2021      9     -2.996145e-07
7317              Po  2021     10      2.185397e-07
7318              Po  2021     11     -6.295571e-08
7319              Po  2021     12     -2.250248e-08

[7320 rows x 4 columns]


### V- TEMPERATURE DE POINT DE ROSÉE

In [28]:
# Lecture de fichier de vent u à la 700 hPa
t_point_rosee=Dataset(str(Config.ERA5_PATH) + '/' + str(Config.POINT_ROSEE))
#Lecture des variables du fichier au format netCDF
point_rosee=t_point_rosee.variables['d2m'][:]
unitd=t_point_rosee.variables['d2m'].units
unit=t_point_rosee.variables['time'].units
times=t_point_rosee.variables['time'][:]
lat=t_point_rosee.variables['latitude'][:]
lon=t_point_rosee.variables['longitude'][:]

In [29]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_out = Config.ERA5_DATA_PATH
os.makedirs(dir_out, exist_ok=True)

# Créer une liste pour stocker les informations des stations
station_data_list = []

for index, value in stations.iterrows():
    station = value['station']  
    
    lon_point = value['longitude']  # Assurez-vous que la colonne s'appelle 'Longitude'
    lat_point = value['latitude']   # Assurez-vous que la colonne s'appelle 'Latitude'
    
    # Calcul des carrés des différences entre les latitudes et longitudes
    sq_diff_lat = (lat - lat_point) ** 2
    sq_diff_lon = (lon - lon_point) ** 2
    
    print(f"Generating time series for {station}")
    ref_date = datetime.datetime(int(unit[12:16]), int(unit[17:19]), int(unit[20:22]))
    date_range = []
    point_rosee_data = []
    
    for index, time in enumerate(times):
        date_time = ref_date + datetime.timedelta(hours=int(time))
        date_range.append(date_time)  # Ajoutez l'objet datetime lui-même
        point_rosee_data.append(point_rosee[index, sq_diff_lat.argmin(), sq_diff_lon.argmin()])  # Utilisez les indices corrects
    station_data_list.append({'Station': station,'Date-Time': date_range, 't_point_rosee': point_rosee_data})

# Création du DataFrame global
all_data = pd.DataFrame()

for station_data in station_data_list:
    df = pd.DataFrame(station_data)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Enregistrement du DataFrame global dans un fichier CSV
output_file_path = os.path.join(dir_out, str(Config.POINT_ROSEE_CSV))
all_data.to_csv(output_file_path, index=False)
print(f"All station data is saved in the CSV file: {output_file_path}")

Generating time series for Bobo_Dioulasso
Generating time series for Bogande
Generating time series for Boromo
Generating time series for Dedougou
Generating time series for Dori
Generating time series for Fada_Ngourma
Generating time series for Gaoua
Generating time series for Ouagadougou_aero
Generating time series for Ouahigouya
Generating time series for Po
All station data is saved in the CSV file: ../ASSETS/DATA/ERA5/point_rosee.csv


In [30]:
all_data.head()

,Station,Date-Time,t_point_rosee
0,Bobo_Dioulasso,1961-01-01 00:00:00,270.459415
1,Bobo_Dioulasso,1961-01-01 06:00:00,270.785245
2,Bobo_Dioulasso,1961-01-01 12:00:00,271.654417
3,Bobo_Dioulasso,1961-01-01 18:00:00,276.274565
4,Bobo_Dioulasso,1961-01-02 00:00:00,274.508269


In [31]:
# Supposons que vous ayez déjà chargé vos données dans un DataFrame appelé 'df'
# Assurez-vous que la colonne 'Date-Time' est en format de datetime
df=all_data.copy()
df['Date-Time'] = pd.to_datetime(df['Date-Time'])
# Créez une colonne 'Decade' en utilisant le numéro de décade dans le mois
decade_ranges = [0, 10, 20, 32]
dek_num = [1, 2, 3]
df['Day']=df['Date-Time'].dt.day
df['Decade'] = pd.cut(df['Day'], bins=decade_ranges, labels=dek_num)
# Créez les colonnes 'Year' et 'Month' à partir de la colonne 'Date-Time'
df['Year'] = df['Date-Time'].dt.year
df['Month'] = df['Date-Time'].dt.month

##### V-1 CALCUL DES MOYENNES DECADAIRES 

In [32]:
# Regroupez par année, mois et décade, puis calculez la moyenne décadaires pour 'd2m (m s**-1)'
decade_avg_pt_ros = df.groupby(['Station','Year', 'Month', 'Decade'])['t_point_rosee'].mean()
# Réinitialisez l'index pour obtenir une DataFrame propre avec les valeurs calculées
decade_avg_pt_ros = decade_avg_pt_ros.reset_index()
#output_file_path = os.path.join(dir_out, str(Config.WIND_V_975_CSV))
output_dek_path = os.path.join(dir_out, str(Config.POINT_ROSEE_CSV_DEK))
decade_avg_pt_ros.to_csv(output_dek_path, index=False)
df_terre_dek.append(decade_avg_pt_ros)
# Affichez les moyennes décadaires
print("Moyenne décadaire :\n", decade_avg_pt_ros)

Moyenne décadaire :
               Station  Year  Month Decade  t_point_rosee
0      Bobo_Dioulasso  1961      1      1     274.869892
1      Bobo_Dioulasso  1961      1      2     274.777057
2      Bobo_Dioulasso  1961      1      3     274.205329
3      Bobo_Dioulasso  1961      2      1     270.083815
4      Bobo_Dioulasso  1961      2      2     269.565937
...               ...   ...    ...    ...            ...
21955              Po  2021     11      2     289.186040
21956              Po  2021     11      3     285.818809
21957              Po  2021     12      1     282.116991
21958              Po  2021     12      2     279.155191
21959              Po  2021     12      3     275.489273

[21960 rows x 5 columns]


##### V-2 CALCUL DES MOYENNES MENSUELLES

In [33]:
# Maintenant, calculez la moyenne mensuelle en utilisant le même processus
month_avg_pt_ros = df.groupby(['Station','Year', 'Month'])['t_point_rosee'].mean()
month_avg_pt_ros = month_avg_pt_ros.reset_index()
# Affichez les moyennes mensuelles
print("\nMoyenne mensuelle :\n", month_avg_pt_ros)
output_mon_path = os.path.join(dir_out, str(Config.POINT_ROSEE_CSV_MON))
month_avg_pt_ros.to_csv(output_mon_path, index=False)
df_terre_mon.append(month_avg_pt_ros)


Moyenne mensuelle :
              Station  Year  Month  t_point_rosee
0     Bobo_Dioulasso  1961      1     274.604133
1     Bobo_Dioulasso  1961      2     270.981565
2     Bobo_Dioulasso  1961      3     276.209542
3     Bobo_Dioulasso  1961      4     291.349840
4     Bobo_Dioulasso  1961      5     294.730945
...              ...   ...    ...            ...
7315              Po  2021      8     296.557268
7316              Po  2021      9     296.768992
7317              Po  2021     10     296.601037
7318              Po  2021     11     289.805219
7319              Po  2021     12     278.809800

[7320 rows x 4 columns]


### VI- HUMIDITE DU SOL

In [34]:
# Lecture de fichier de vent u à la 700 hPa
h_vol_sol_wat=Dataset(str(Config.ERA5_PATH) + '/' + str(Config.SOL_WATERPOINT))

In [35]:
# Lecture de fichier de vent u à la 700 hPa
h_vol_sol_wat=Dataset(str(Config.ERA5_PATH) + '/' + str(Config.SOL_WATERPOINT))
#Lecture des variables du fichier au format netCDF
volum_wat=h_vol_sol_wat.variables['swvl1'][:]
unitd=h_vol_sol_wat.variables['swvl1'].units
unit=h_vol_sol_wat.variables['time'].units
times=h_vol_sol_wat.variables['time'][:]
lat=h_vol_sol_wat.variables['latitude'][:]
lon=h_vol_sol_wat.variables['longitude'][:]

In [36]:
# Créer un répertoire de sortie s'il n'existe pas déjà
dir_out = Config.ERA5_DATA_PATH
os.makedirs(dir_out, exist_ok=True)

# Créer une liste pour stocker les informations des stations
station_data_list = []

for index, value in stations.iterrows():
    station = value['station']  
    
    lon_point = value['longitude']  # Assurez-vous que la colonne s'appelle 'Longitude'
    lat_point = value['latitude']   # Assurez-vous que la colonne s'appelle 'Latitude'
    
    # Calcul des carrés des différences entre les latitudes et longitudes
    sq_diff_lat = (lat - lat_point) ** 2
    sq_diff_lon = (lon - lon_point) ** 2
    
    print(f"Generating time series for {station}")
    ref_date = datetime.datetime(int(unit[12:16]), int(unit[17:19]), int(unit[20:22]))
    date_range = []
    volum_wat_data = []
    
    for index, time in enumerate(times):
        date_time = ref_date + datetime.timedelta(hours=int(time))
        date_range.append(date_time)  # Ajoutez l'objet datetime lui-même
        volum_wat_data.append(volum_wat[index, sq_diff_lat.argmin(), sq_diff_lon.argmin()])  # Utilisez les indices corrects
    station_data_list.append({'Station': station,'Date-Time': date_range, 'h_vol_sol_wat': volum_wat_data})

# Création du DataFrame global
all_data = pd.DataFrame()

for station_data in station_data_list:
    df = pd.DataFrame(station_data)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Enregistrement du DataFrame global dans un fichier CSV
output_file_path = os.path.join(dir_out, str(Config.SOL_WATERPOINT_CSV))
all_data.to_csv(output_file_path, index=False)
print(f"All station data is saved in the CSV file: {output_file_path}")

Generating time series for Bobo_Dioulasso
Generating time series for Bogande
Generating time series for Boromo
Generating time series for Dedougou
Generating time series for Dori
Generating time series for Fada_Ngourma
Generating time series for Gaoua
Generating time series for Ouagadougou_aero
Generating time series for Ouahigouya
Generating time series for Po
All station data is saved in the CSV file: ../ASSETS/DATA/ERA5/vol_soil_water.csv


In [37]:
all_data.head()

,Station,Date-Time,h_vol_sol_wat
0,Bobo_Dioulasso,1961-01-01 00:00:00,0.167690
1,Bobo_Dioulasso,1961-01-01 06:00:00,0.167649
2,Bobo_Dioulasso,1961-01-01 12:00:00,0.167177
3,Bobo_Dioulasso,1961-01-01 18:00:00,0.166498
4,Bobo_Dioulasso,1961-01-02 00:00:00,0.177357


In [38]:
# Supposons que vous ayez déjà chargé vos données dans un DataFrame appelé 'df'
# Assurez-vous que la colonne 'Date-Time' est en format de datetime
df=all_data.copy()
df['Date-Time'] = pd.to_datetime(df['Date-Time'])
# Créez une colonne 'Decade' en utilisant le numéro de décade dans le mois
decade_ranges = [0, 10, 20, 32]
dek_num = [1, 2, 3]
df['Day']=df['Date-Time'].dt.day
df['Decade'] = pd.cut(df['Day'], bins=decade_ranges, labels=dek_num)
# Créez les colonnes 'Year' et 'Month' à partir de la colonne 'Date-Time'
df['Year'] = df['Date-Time'].dt.year
df['Month'] = df['Date-Time'].dt.month

##### VI-1 CALCUL DES MOYENNES DECADAIRES 

In [39]:
# Regroupez par année, mois et décade, puis calculez la moyenne décadaires pour 'd2m (m s**-1)'
decade_avg_wat_sol = df.groupby(['Station','Year', 'Month', 'Decade'])['h_vol_sol_wat'].mean()
# Réinitialisez l'index pour obtenir une DataFrame propre avec les valeurs calculées
decade_avg_wat_sol = decade_avg_wat_sol.reset_index()
#output_file_path = os.path.join(dir_out, str(Config.WIND_V_975_CSV))
output_dek_path = os.path.join(dir_out, str(Config.SOL_WATERPOINT_CSV_DEK))
decade_avg_wat_sol.to_csv(output_dek_path, index=False)

# Affichez les moyennes décadaires
print("Moyenne décadaire :\n", decade_avg_wat_sol)
df_terre_dek.append(decade_avg_wat_sol)

Moyenne décadaire :
               Station  Year  Month Decade  h_vol_sol_wat
0      Bobo_Dioulasso  1961      1      1       0.177988
1      Bobo_Dioulasso  1961      1      2       0.178784
2      Bobo_Dioulasso  1961      1      3       0.178600
3      Bobo_Dioulasso  1961      2      1       0.172708
4      Bobo_Dioulasso  1961      2      2       0.173695
...               ...   ...    ...    ...            ...
21955              Po  2021     11      2       0.131637
21956              Po  2021     11      3       0.124747
21957              Po  2021     12      1       0.123271
21958              Po  2021     12      2       0.120752
21959              Po  2021     12      3       0.117617

[21960 rows x 5 columns]


##### VI-2 CALCUL DES MOYENNES MENSUELLES

In [40]:
# Maintenant, calculez la moyenne mensuelle en utilisant le même processus
month_avg_wat_sol = df.groupby(['Station','Year', 'Month'])['h_vol_sol_wat'].mean()
month_avg_wat_sol = month_avg_wat_sol.reset_index()
# Affichez les moyennes mensuelles
print("\nMoyenne mensuelle :\n", month_avg_wat_sol)
output_mon_path = os.path.join(dir_out, str(Config.SOL_WATERPOINT_CSV_MON))
month_avg_wat_sol.to_csv(output_mon_path, index=False)
df_terre_mon.append(month_avg_wat_sol)


Moyenne mensuelle :
              Station  Year  Month  h_vol_sol_wat
0     Bobo_Dioulasso  1961      1       0.178462
1     Bobo_Dioulasso  1961      2       0.173626
2     Bobo_Dioulasso  1961      3       0.170209
3     Bobo_Dioulasso  1961      4       0.204703
4     Bobo_Dioulasso  1961      5       0.262517
...              ...   ...    ...            ...
7315              Po  2021      8       0.397414
7316              Po  2021      9       0.323568
7317              Po  2021     10       0.217716
7318              Po  2021     11       0.134100
7319              Po  2021     12       0.120452

[7320 rows x 4 columns]


### VII- PROCESSING LANGUE D'EAU FROIDE DATA

In [41]:
input_directory =Config.SST_LEF_PATH
output_directory =Config.SST_LEF_SP_MN_PATH

##### Calcul de la moyenne spatiale

In [42]:
# Dossier d'entrée contenant les fichiers SST
input_directory =Config.SST_LEF_PATH

# Dossier de sortie pour les fichiers résultants
output_directory = Config.SST_LEF_SP_MN_PATH
#os.makedirs(output_directory, exist_ok=True)

if os.path.exists(output_directory):
    # S'il existe, supprimez-le
    shutil.rmtree(output_directory)
# Maintenant, recréez le dossier
os.makedirs(output_directory)



# Boucle sur les fichiers dans le dossier d'entrée
for filename in os.listdir(input_directory):
    if filename.endswith('.nc'):
        # Chemin complet vers le fichier d'entrée
        input_file = os.path.join(input_directory, filename)
        
        # Déterminez le nom du fichier de sortie en ajoutant "_fldmean" au nom d'origine
        output_filename = os.path.splitext(filename)[0] + '_fldmean.nc'
        output_file = os.path.join(output_directory, output_filename)
        
        # Exécutez la commande CDO pour calculer la moyenne spatiale (fldmean)
        cdo_command = f"cdo fldmean {input_file} {output_file}"
        
        # Exécutez la commande en utilisant subprocess
        subprocess.run(cdo_command, shell=True)

print("Calcul de la moyenne spatiale (fldmean) terminé.")

Calcul de la moyenne spatiale (fldmean) terminé.


##### Concatenation des fichiers après moyenne spatiale

In [43]:
# Nom du fichier de sortie concaténé
output_merged_file = os.path.join(output_directory, Config.SST_LEF_MERGE)

# Liste de fichiers de sortie à concaténer
output_files_to_concatenate = [os.path.join(output_directory, filename) for filename in os.listdir(output_directory) if filename.endswith('_fldmean.nc')]

# Vérifiez s'il y a au moins un fichier à concaténer
if len(output_files_to_concatenate) > 0:
    # Utilisez CDO pour concaténer les fichiers
    cdo_command = f"cdo mergetime {' '.join(output_files_to_concatenate)} {output_merged_file}"
    subprocess.run(cdo_command, shell=True)
    print(f"Concaténation terminée. Le fichier concaténé est enregistré sous {output_merged_file}")
else:
    print("Aucun fichier de sortie à concaténer trouvé dans le répertoire de sortie.")

Concaténation terminée. Le fichier concaténé est enregistré sous ../ASSETS/DATA/RAW/ERA5/SST_LEF_FLDMEAN/sst_leg_merged.nc


##### Transformation du fichier netcdf en dataframe

In [44]:
# Ouvrir le fichier NetCDF
data = netCDF4.Dataset(output_merged_file)
# Extraire les variables time et sst
time_values = data.variables['time'][:]
sst_values = data.variables['sst'][:]

# Redimensionner les tableaux pour les rendre 1D
time_values = time_values.flatten()
sst_values = sst_values.flatten()
# Extraire les attributs de la variable "time"
time_var = data.variables['time']
units = time_var.units  # Obtenez l'unité de temps
# Fermer le fichier NetCDF
data.close()

# Convertir les valeurs de "time" en dates en utilisant cftime
dates = cftime.num2date(time_values, units, calendar='standard')

# Créer un DataFrame Pandas avec les dates converties
df_merged_sst_lef = pd.DataFrame({'date': dates, 'sst': sst_values})

In [45]:
# Utilisez apply pour convertir chaque valeur en datetime
df_merged_sst_lef['date'] = df_merged_sst_lef['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if isinstance(x, cftime._cftime.DatetimeGregorian) else x)
df_merged_sst_lef['date'] = pd.to_datetime(df_merged_sst_lef['date'])

In [46]:
df_merged_sst_lef['Year'] = df_merged_sst_lef['date'].dt.year
# Créez la colonne 'Mois' en extrayant le mois
df_merged_sst_lef['Month'] = df_merged_sst_lef['date'].dt.month
# Créez la colonne 'Jour' en extrayant le jour
df_merged_sst_lef['Day'] = df_merged_sst_lef['date'].dt.day
df_merged_sst_lef['Decade']=pd.cut(df_merged_sst_lef['Day'],bins=[0,10,20,32], labels=[1,2,3])

#### VII.1 - CALCUL DECADAIRE

In [47]:
df_sst_lef_dek=df_merged_sst_lef.copy()
# Utilisez la méthode groupby pour regrouper les données par 'Annee', 'Mois' et 'Decade' et calculez la moyenne de 'sst' pour chaque groupe
df_sst_lef_dek['Moy_lef_Dec'] = df_sst_lef_dek.groupby(['Year', 'Month', 'Decade'])['sst'].transform('mean')
# Si vous souhaitez supprimer les lignes en double après avoir ajouté la colonne 'Moy_Dec', vous pouvez le faire en utilisant la méthode drop_duplicates
df_sst_lef_dek.drop_duplicates(subset=['Year', 'Month', 'Decade', 'Moy_lef_Dec'], inplace=True)
# Affichez le DataFrame résultant
df_sst_lef_dek.head()

,date,sst,Year,Month,Day,Decade,Moy_lef_Dec
0,1961-01-01,300.468903,1961,1,1,1,300.383972
80,1961-01-11,300.123718,1961,1,11,2,300.250214
160,1961-01-21,300.506195,1961,1,21,3,300.686920
248,1961-02-01,300.969025,1961,2,1,1,300.884918
328,1961-02-11,300.888519,1961,2,11,2,301.116699


##### Calcul anomalies

In [48]:
# Filtrer le DataFrame pour les années 1961-1990
ref_start=1961
ref_end=1991
normal_period = df_sst_lef_dek[(df_sst_lef_dek['Year'] >= ref_start) & (df_sst_lef_dek['Year'] <= ref_end)]
# Calculer la moyenne de la colonne 'sst' pour cette période
sst_mean = normal_period['sst'].mean()
# Afficher la moyenne
print("Moyenne pour la période 1961-1990 :", sst_mean)
df_sst_lef_dek['anom_lef_dek']=df_sst_lef_dek['Moy_lef_Dec']-sst_mean
df_sst_lef_dek=df_sst_lef_dek[['Year','Month','Decade','anom_lef_dek']]
df_mer_dek.append(df_sst_lef_dek)

Moyenne pour la période 1961-1990 : 300.0179


In [49]:
df_sst_lef_dek.head()

,Year,Month,Decade,anom_lef_dek
0,1961,1,1,0.366058
80,1961,1,2,0.232300
160,1961,1,3,0.669006
248,1961,2,1,0.867004
328,1961,2,2,1.098785


#### I.2 - CALCUL MENSUEL

In [50]:
df_sst_lef_mois=df_merged_sst_lef.copy()
# Utilisez la méthode groupby pour regrouper les données par 'Annee', 'Mois' et 'Decade' et calculez la moyenne de 'sst' pour chaque groupe
df_sst_lef_mois['Moy_lef_Mois'] = df_sst_lef_mois.groupby(['Year', 'Month'])['sst'].transform('mean')
# Si vous souhaitez supprimer les lignes en double après avoir ajouté la colonne 'Moy_Dec', vous pouvez le faire en utilisant la méthode drop_duplicates
df_sst_lef_mois.drop_duplicates(subset=['Year', 'Month', 'Moy_lef_Mois'], inplace=True)
# Affichez le DataFrame résultant
df_sst_lef_mois.head()

,date,sst,Year,Month,Day,Decade,Moy_lef_Mois
0,1961-01-01,300.468903,1961,1,1,1,300.448334
248,1961-02-01,300.969025,1961,2,1,1,301.100372
472,1961-03-01,301.283325,1961,3,1,1,301.556671
720,1961-04-01,301.666687,1961,4,1,1,301.702454
960,1961-05-01,301.620117,1961,5,1,1,301.258148


##### Calcul anomalies mensuelles

In [51]:
# Filtrer le DataFrame pour les années 1961-1990
ref_start=1961
ref_end=1991
normal_period = df_sst_lef_mois[(df_sst_lef_mois['Year'] >= ref_start) & (df_sst_lef_mois['Year'] <= ref_end)]
# Calculer la moyenne de la colonne 'sst' pour cette période
sst_mean = normal_period['sst'].mean()
# Afficher la moyenne
print("Moyenne pour la période 1961-1990 :", sst_mean)
df_sst_lef_mois['anom_lef_mois']=df_sst_lef_mois['Moy_lef_Mois']-sst_mean
df_sst_lef_mois=df_sst_lef_mois[['Year','Month','anom_lef_mois']]

df_mer_mon.append(df_sst_lef_mois)

Moyenne pour la période 1961-1990 : 300.01572


In [52]:
df_sst_lef_mois.head()

,Year,Month,anom_lef_mois
0,1961,1,0.432617
248,1961,2,1.084656
472,1961,3,1.540955
720,1961,4,1.686737
960,1961,5,1.242432


### VIII- PROCESSING NINO 3.4 DATA

##### Calcul de la moyenne spatiale

In [53]:
# Dossier d'entrée contenant les fichiers SST
input_directory =Config.SST_NINO_PATH

# Dossier de sortie pour les fichiers résultants
output_directory = Config.SST_NINO_SP_MN_PATH
#os.makedirs(output_directory, exist_ok=True)
if os.path.exists(output_directory):
    # S'il existe, supprimez-le
    shutil.rmtree(output_directory)
# Maintenant, recréez le dossier
os.makedirs(output_directory)

# Boucle sur les fichiers dans le dossier d'entrée
for filename in os.listdir(input_directory):
    if filename.endswith('.nc'):
        # Chemin complet vers le fichier d'entrée
        input_file = os.path.join(input_directory, filename)
        
        # Déterminez le nom du fichier de sortie en ajoutant "_fldmean" au nom d'origine
        output_filename = os.path.splitext(filename)[0] + '_fldmean.nc'
        output_file = os.path.join(output_directory, output_filename)
        
        # Exécutez la commande CDO pour calculer la moyenne spatiale (fldmean)
        cdo_command = f"cdo fldmean {input_file} {output_file}"
        
        # Exécutez la commande en utilisant subprocess
        subprocess.run(cdo_command, shell=True)

print("Calcul de la moyenne spatiale (fldmean) terminé.")

Calcul de la moyenne spatiale (fldmean) terminé.


##### Concatenation des fichiers après moyenne spatiale

In [54]:
# Nom du fichier de sortie concaténé
output_merged_file = os.path.join(output_directory, Config.SST_NINO_MERGE)

# Liste de fichiers de sortie à concaténer
output_files_to_concatenate = [os.path.join(output_directory, filename) for filename in os.listdir(output_directory) if filename.endswith('_fldmean.nc')]

# Vérifiez s'il y a au moins un fichier à concaténer
if len(output_files_to_concatenate) > 0:
    # Utilisez CDO pour concaténer les fichiers
    cdo_command = f"cdo mergetime {' '.join(output_files_to_concatenate)} {output_merged_file}"
    subprocess.run(cdo_command, shell=True)
    print(f"Concaténation terminée. Le fichier concaténé est enregistré sous {output_merged_file}")
else:
    print("Aucun fichier de sortie à concaténer trouvé dans le répertoire de sortie.")

Concaténation terminée. Le fichier concaténé est enregistré sous ../ASSETS/DATA/RAW/ERA5/SST_NINO_FLDMEAN/sst_nino_merged.nc


##### Transformation du fichier netcdf en dataframe

In [55]:
# Ouvrir le fichier NetCDF
data = netCDF4.Dataset(output_merged_file)
# Extraire les variables time et sst
time_values = data.variables['time'][:]
sst_values = data.variables['sst'][:]

# Redimensionner les tableaux pour les rendre 1D
time_values = time_values.flatten()
sst_values = sst_values.flatten()
# Extraire les attributs de la variable "time"
time_var = data.variables['time']
units = time_var.units  # Obtenez l'unité de temps
# Fermer le fichier NetCDF
data.close()

# Convertir les valeurs de "time" en dates en utilisant cftime
dates = cftime.num2date(time_values, units, calendar='standard')

# Créer un DataFrame Pandas avec les dates converties
df_merged_sst_nino = pd.DataFrame({'date': dates, 'sst': sst_values})

In [56]:
# Utilisez apply pour convertir chaque valeur en datetime
df_merged_sst_nino['date'] = df_merged_sst_nino['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if isinstance(x, cftime._cftime.DatetimeGregorian) else x)
df_merged_sst_nino['date'] = pd.to_datetime(df_merged_sst_nino['date'])

In [57]:
df_merged_sst_nino['Year'] = df_merged_sst_nino['date'].dt.year
# Créez la colonne 'Mois' en extrayant le mois
df_merged_sst_nino['Month'] = df_merged_sst_nino['date'].dt.month
# Créez la colonne 'Jour' en extrayant le jour
df_merged_sst_nino['Day'] = df_merged_sst_nino['date'].dt.day
df_merged_sst_nino['Decade']=pd.cut(df_merged_sst_nino['Day'],bins=[0,10,20,32], labels=[1,2,3])

#### VIII.1 - CALCUL DECADAIRE

In [58]:
df_sst_nino_dek=df_merged_sst_nino.copy()
# Utilisez la méthode groupby pour regrouper les données par 'Annee', 'Mois' et 'Decade' et calculez la moyenne de 'sst' pour chaque groupe
df_sst_nino_dek['Moy_nino_Dec'] = df_sst_nino_dek.groupby(['Year', 'Month','Decade'])['sst'].transform('mean')
# Si vous souhaitez supprimer les lignes en double après avoir ajouté la colonne 'Moy_Dec', vous pouvez le faire en utilisant la méthode drop_duplicates
df_sst_nino_dek.drop_duplicates(subset=['Year', 'Month','Decade','Moy_nino_Dec'], inplace=True, keep='last')
# Affichez le DataFrame résultant
#df_sst_nino_dek.head()

##### Calcul anomalies decadaires

In [59]:
# Filtrer le DataFrame pour les années 1961-1990
ref_start=1961
ref_end=1991
normal_period = df_sst_nino_dek[(df_sst_nino_dek['Year'] >= ref_start) & (df_sst_nino_dek['Year'] <= ref_end)]
# Calculer la moyenne de la colonne 'sst' pour cette période
sst_mean = normal_period['sst'].mean()
# Afficher la moyenne
print("Moyenne pour la période 1961-1990 :", sst_mean)
df_sst_nino_dek['anom_nino_dek']=df_sst_nino_dek['Moy_nino_Dec']-sst_mean
df_sst_nino_dek=df_sst_nino_dek[['Year','Month','Decade','anom_nino_dek']]
df_mer_dek.append(df_sst_nino_dek)

Moyenne pour la période 1961-1990 : 300.7522


#### VIII.2 - CALCUL MENSUEL

In [60]:
df_sst_nino_mois=df_merged_sst_nino.copy()
# Utilisez la méthode groupby pour regrouper les données par 'Annee', 'Mois' et 'Decade' et calculez la moyenne de 'sst' pour chaque groupe
df_sst_nino_mois['Moy_nino_Mois'] = df_sst_nino_mois.groupby(['Year', 'Month'])['sst'].transform('mean')
# Si vous souhaitez supprimer les lignes en double après avoir ajouté la colonne 'Moy_Dec', vous pouvez le faire en utilisant la méthode drop_duplicates
df_sst_nino_mois.drop_duplicates(subset=['Year', 'Month','Moy_nino_Mois'], inplace=True, keep='last')
# Affichez le DataFrame résultant
df_sst_nino_mois.head()

,date,sst,Year,Month,Day,Decade,Moy_nino_Mois
247,1961-01-31 21:00:00,301.076355,1961,1,31,3,300.850891
471,1961-02-28 21:00:00,301.244873,1961,2,28,3,301.184143
719,1961-03-31 21:00:00,301.603821,1961,3,31,3,301.383850
959,1961-04-30 21:00:00,301.675903,1961,4,30,3,301.779083
1207,1961-05-31 21:00:00,301.377045,1961,5,31,3,301.479736


In [61]:
# Filtrer le DataFrame pour les années 1961-1990
ref_start=1961
ref_end=1991
normal_period = df_sst_nino_mois[(df_sst_nino_mois['Year'] >= ref_start) & (df_sst_nino_mois['Year'] <= ref_end)]
# Calculer la moyenne de la colonne 'sst' pour cette période
sst_mean = normal_period['sst'].mean()
# Afficher la moyenne
print("Moyenne pour la période 1961-1990 :", sst_mean)
df_sst_nino_mois['anom_nino_mois']=df_sst_nino_mois['Moy_nino_Mois']-sst_mean
df_sst_nino_mois=df_sst_nino_mois[['Year','Month','anom_nino_mois']]
df_mer_mon.append(df_sst_nino_mois)

Moyenne pour la période 1961-1990 : 300.75037


### IX- Récapitulation et sauvegarde des dataframes

##### Fusion des dataframes des paramètres locaux (mesurés sur le Burkina)

##### Decadaires

In [62]:
# Fusion des dataframes de la liste df_terre_dek
df_terre_merg_dek = reduce(lambda left, right: pd.merge(left, right, on=['Station', 'Year', 'Month', 'Decade']), df_terre_dek)

# Afficher le dataframe fusionné
print(df_terre_merg_dek)
#df_terre_merg_dek.to_csv('./test_fusion.xlsx')

              Station  Year  Month Decade  v_wind_975  u_wind_700  u_wind_100  \
0      Bobo_Dioulasso  1961      1      1   -2.477929   -6.607575   16.585018   
1      Bobo_Dioulasso  1961      1      2   -2.545220   -1.387879   18.501326   
2      Bobo_Dioulasso  1961      1      3   -2.898044    2.169399   17.749175   
3      Bobo_Dioulasso  1961      2      1   -2.424871    3.171007    9.010893   
4      Bobo_Dioulasso  1961      2      2   -2.312341    3.065763   11.091734   
...               ...   ...    ...    ...         ...         ...         ...   
21955              Po  2021     11      2   -0.261836   -5.803122    7.834724   
21956              Po  2021     11      3   -0.620669   -8.759756    5.852299   
21957              Po  2021     12      1   -1.527897   -4.211920   11.448239   
21958              Po  2021     12      2   -2.103897   -5.516121   18.516873   
21959              Po  2021     12      3   -3.658856   -1.577647   17.013145   

       eau_precipitable  t_

##### Mensuelles

In [63]:
# Fusion des dataframes de la liste df_terre_dek
df_terre_merg_mon = reduce(lambda left, right: pd.merge(left, right, on=['Station', 'Year', 'Month']), df_terre_mon)

# Afficher le dataframe fusionné
print(df_terre_merg_mon)
#df_terre_merg_mon.to_csv('./test_merg_terre_mon.xlsx')

             Station  Year  Month  v_wind_975  u_wind_700  u_wind_100  \
0     Bobo_Dioulasso  1961      1   -2.648709   -1.809392   17.616270   
1     Bobo_Dioulasso  1961      2   -2.103444    0.458890   12.566036   
2     Bobo_Dioulasso  1961      3   -0.886015   -1.004652   16.747609   
3     Bobo_Dioulasso  1961      4    1.652940   -4.718261    5.372466   
4     Bobo_Dioulasso  1961      5    1.654241   -8.676686   -4.629670   
...              ...   ...    ...         ...         ...         ...   
7315              Po  2021      8    1.879763   -6.685376  -15.133298   
7316              Po  2021      9    2.036198  -11.709194   -8.326725   
7317              Po  2021     10    1.988795  -10.225080   -2.909251   
7318              Po  2021     11    0.455663   -7.267599    7.878928   
7319              Po  2021     12   -2.469851   -3.697888   15.703087   

      eau_precipitable  t_point_rosee  h_vol_sol_wat  
0        -2.787611e-07     274.604133       0.178462  
1        -4.1

##### Fusion des dataframes des anomalies des températures de mers

##### Decadaire

In [64]:
# Fusion des dataframes de la liste df_terre_dek
df_mer_merg_dek = reduce(lambda left, right: pd.merge(left, right, on=['Year', 'Month', 'Decade']), df_mer_dek)
# Afficher le dataframe fusionné
print(df_mer_merg_dek)
#df_mer_merg_dek.to_csv('./test_fus_mer_dek.xlsx')

      Year  Month Decade  anom_lef_dek  anom_nino_dek
0     1961      1      1      0.366058      -0.024109
1     1961      1      2      0.232300       0.064850
2     1961      1      3      0.669006       0.241150
3     1961      2      1      0.867004       0.408600
4     1961      2      2      1.098785       0.390625
...    ...    ...    ...           ...            ...
2191  2021     11      2      0.847351       0.153992
2192  2021     11      3      0.882965       0.132477
2193  2021     12      1      0.773315       0.054108
2194  2021     12      2      0.721924       0.027649
2195  2021     12      3      0.703491       0.093109

[2196 rows x 5 columns]


##### Mensuelle

In [65]:
# Fusion des dataframes de la liste df_terre_dek
df_mer_merg_mon = reduce(lambda left, right: pd.merge(left, right, on=['Year', 'Month']), df_mer_mon)
# Afficher le dataframe fusionné
print(df_mer_merg_mon)
#df_mer_merg_mon.to_csv('./test_fus_mer_mon.xlsx')

     Year  Month  anom_lef_mois  anom_nino_mois
0    1961      1       0.432617        0.100525
1    1961      2       1.084656        0.433777
2    1961      3       1.540955        0.633484
3    1961      4       1.686737        1.028717
4    1961      5       1.242432        0.729370
..    ...    ...            ...             ...
727  2021      8      -0.499146       -0.114014
728  2021      9      -0.291443       -0.145203
729  2021     10       0.351349        0.091980
730  2021     11       0.799896        0.187561
731  2021     12       0.734161        0.061249

[732 rows x 4 columns]


##### IX-1 Fusion des dataframes locaux et mer

##### Decadaires

In [66]:
# Fusionner les DataFrames en utilisant une fusion externe (outer) sans spécifier les colonnes de fusion
df_dek = df_terre_merg_dek.merge(df_mer_merg_dek, how='outer')
df_dek.to_csv(str(Config.FILES_TRAITED_PATH) + '/' + str(Config.ERA_FUSION_DEK),index=False)

##### Mensuelles

In [67]:
# Fusionner les DataFrames en utilisant une fusion externe (outer) sans spécifier les colonnes de fusion
df_mon = df_terre_merg_mon.merge(df_mer_merg_mon, how='outer')
df_mon.to_csv(str(Config.FILES_TRAITED_PATH) + '/' + str(Config.ERA_FUSION_MON),index=False)